In [9]:
import pandas as pd
from stage import fullPrint
from datetime import datetime, timedelta
import yahoo_fin.stock_info as yf
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)

In [ ]:
PEAK = 13

In [10]:
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = (nasdaqList + nyseList)

In [11]:
SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))

now = datetime.now()
monday = now - timedelta(days = now.weekday())
monday = monday.strftime('%Y-%m-%d')

for ticker in ["MRNA"]:
    try:
        df = yf.get_data(ticker, start_date="1995-01-02",end_date = monday, index_as_date = True, interval="1wk")
        df.to_pickle('stockData/nysenasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
import pickle
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
  
except:
    print("Something went wrong")

1/5586


In [12]:
import numpy as np
import yahoo_fin.stock_info as yf
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,4]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])


In [13]:
from stage import fullPrint
import swifter
import yahoo_fin.stock_info as yf
import pandas as pd
import os
spDf = yf.get_data('^GSPC', start_date="1995-01-02",end_date = monday, index_as_date = True, interval="1wk")
for ticker in ["MRNA"]:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        if(stockDf['close'].isnull().values.any()):
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            continue
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, PEAK] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

Pandas Apply: 100%|██████████| 1416/1416 [00:00<00:00, 25342.79it/s]

MRNA


In [14]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-02",end_date= monday, index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [15]:
dfMRNA = pd.read_pickle("stockData/nyseNasdaq/MRNA.pkl")
dfMRNA.to_csv("MRNA.csv")

In [8]:
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass